In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io
import mne
from mne_bids import BIDSPath, write_raw_bids, make_dataset_description
import logging

from extract_scalar import *

In [2]:
# =============================================================================
# Configuration and Constants
# =============================================================================

# Set up logging: messages will include timestamp, log level, and message content
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

DS_ROOT = '/volumes/hyijie_psy/CPP/data_mid_van_Vugt_2019'

PREPROCESS_DATA = '../data/preprocessData'
PATH_PREPROCESS_MEM = os.path.join(PREPROCESS_DATA,'Memory')
PATH_PREPROCESS_PERC = os.path.join(PREPROCESS_DATA,'Perception')

os.makedirs(PREPROCESS_DATA, exist_ok=True)
os.makedirs(PATH_PREPROCESS_MEM, exist_ok=True)
os.makedirs(PATH_PREPROCESS_PERC, exist_ok=True)

SAMPLING_RATE = 500
RT_OUTLIER_STD_MULTIPLIER = 3

EVENT_ID_MAP = {
    'response': 1 
}

SUBJECT_IDS  = [f'ACC{i:03}' for i in range(14, 37)]  # ACC014 to ACC036


In [ ]:
# ==============================
# Helper Functions
# ==============================
def load_behavior_data(sub_id, task_mode, path = DS_ROOT):
    """Load and parse behavior events.mat into a clean DataFrame and save clean one as BIDS."""
    path_behavior = os.path.join(path, sub_id, 'events.mat')
    assert os.path.exists(path_behavior), f"events.mat not found: {path_behavior}"
    raw_behavior = scipy.io.loadmat(path_behavior)['events'][['mode', 'type', 'rt','correct', 'item1', 'item2','cueitem']]

    data_behavior = pd.DataFrame({
        'mode': [extract_scalar(x) for x in raw_behavior['mode'][0]],
        'type': [extract_scalar(x) for x in raw_behavior['type'][0]],
        'rt': [extract_scalar(x) for x in raw_behavior['rt'][0]],
        'acc': [extract_scalar(x) for x in raw_behavior['correct'][0]],
        'item_1': [extract_scalar(x) for x in raw_behavior['item1'][0]],
        'item_2': [extract_scalar(x) for x in raw_behavior['item2'][0]],
        'item_cue': [extract_scalar(x) for x in raw_behavior['cueitem'][0]],
    })

    # Chose tpye and mode of bahevior data and calculate index for a certain mode
    data_behavior_resp = data_behavior[data_behavior['type'] == 'resp'].reset_index(drop=True)
    data_behavior_resp_mode = data_behavior_resp[data_behavior_resp['mode'] == task_mode]
    index_mode_unique = data_behavior_resp_mode.index
    data_behavior_resp_mode = data_behavior_resp_mode.reset_index(drop=True)

    # Set path to save behavior data which was constructed as BIDS
    if task_mode == 'mem':
        dir_data_behavior = os.path.join(PATH_PREPROCESS_MEM, 'sub-'+sub_id,'beh')
    else:
        dir_data_behavior = os.path.join(PATH_PREPROCESS_PERC, 'sub-'+sub_id,'beh')
    
    os.makedirs(dir_data_behavior, exist_ok=True)

    if task_mode == 'mem':
        path_data_behavior = os.path.join(dir_data_behavior,'sub-'+sub_id+'_task-memoryBasedDecisionMaking_beh.tsv')
    else:
        path_data_behavior = os.path.join(dir_data_behavior,'sub-'+sub_id+'_task-perceptionBasedDecisionMaking_beh.tsv')
    
    # Save behavior data into BIDS version
    data_behavior_resp_mode.to_csv(path_data_behavior, sep='\t', index=False, na_rep='n/a')

    logger.info(f"Subject {sub_id}: behavioral data of {task_mode} has been loaded and saved as BIDS.")

    return data_behavior_resp_mode, index_mode_unique

def load_eeg_data(sub_id, index_mode_unique, path = DS_ROOT):
    """Load EEG trial data from .mat file."""
    path_eeg = os.path.join(path, sub_id, f'{sub_id}.mat')
    assert os.path.exists(path_eeg), f"EEG data file not found: {path_eeg}"

    # Load eeg data from .mat files 
    raw_eeg = scipy.io.loadmat(path_eeg)
    data_eeg_mat = raw_eeg['trialdata'][0]['trial'][0]  
    data_eeg_mode_mat = data_eeg_mat[0,index_mode_unique]              # shape: (1, n_trials)
    data_time = raw_eeg['trialdata'][0]['time'][0]       
    data_time_mode = data_time[0, index_mode_unique]                   # shape: (1, n_trials)

    # Construct eeg data into continious formation
    data_eeg_mode_BIDS = np.concatenate(data_eeg_mode_mat, axis=1)

    logger.info(f"Subject {sub_id}: eeg and time have been loaded.")
    return data_eeg_mode_BIDS, data_time_mode

def calculate_events(sub_id, index_mode_unique, task_mode, path=DS_ROOT):
    _, data_time_mode = load_eeg_data(sub_id, index_mode_unique, path)
    n_trial = len(data_time_mode)
    events = np.empty((n_trial,3), dtype=object)
    
    n_timestamps_total = 0 # Initial timestamps
    for trial in range(n_trial):
        data_time_per_trial = data_time_mode[trial]
        n_timestamps = data_time_per_trial.shape[1]
        n_timestamps_total = n_timestamps_total + n_timestamps
        trigger = n_timestamps_total - 0.1*SAMPLING_RATE # The preprocessed data epoched 0.1s interval after response 
        
        events[trial,0] = trigger
        events[trial,1] = 0
        events[trial,2] = 1
    logger.info(f"Get events from subject {sub_id} in {task_mode}  ")
    return events
    
def load_channel_name(sub_id, path=DS_ROOT):
    path_eeg = os.path.join(path, sub_id, f'{sub_id}.mat')
    raw_eeg = scipy.io.loadmat(path_eeg)
    info_channel = raw_eeg['trialdata'][0]['label'][0]
    info_channel_cleaned = [x.item() for x in info_channel.flatten()]

    return info_channel_cleaned  

def process_into_BIDS(sub_id, task_mode, path=DS_ROOT):
    data_behavior_resp_mode, index_mode_unique = load_behavior_data(sub_id, task_mode, path)
    data_eeg_mode_BIDS, _ = load_eeg_data(sub_id, index_mode_unique, path)
    events = calculate_events(sub_id, index_mode_unique, task_mode)
    info_channel = load_channel_name(sub_id, path)

    n_data_behavior_resp_mode = len(data_behavior_resp_mode)
    n_events = len(events)
    
    # Confirm that the number of response events matches the number of behavioral trials
    assert n_data_behavior_resp_mode == n_events, f"Trial count mismatch! EEG: {n_events}, Behavior: {n_data_behavior_resp_mode}"

    # Prepare information for BIDS
    info = mne.create_info(ch_names=info_channel, sfreq=SAMPLING_RATE, ch_types='eeg')
    data_eeg_bids = mne.io.RawArray(data_eeg_mode_BIDS, info)

    # Write BIDS                                    
    if task_mode == 'mem':
        bids_path = BIDSPath(
            subject=sub_id,
            task='memoryBasedDecisionMaking',
            datatype='eeg',
            root=PATH_PREPROCESS_MEM
        )
    else:
        bids_path = BIDSPath(
            subject=sub_id,
            task='perceptionBasedDecisionMaking',
            datatype='eeg',
            root=PATH_PREPROCESS_PERC
        )
    
    write_raw_bids(
        raw=data_eeg_bids,
        bids_path=bids_path,
        events=events,
        event_id=EVENT_ID_MAP,
        format='BrainVision',
        allow_preload=True,
        overwrite=True,
        verbose=False
    )

In [4]:
# ==============================
# Main Execution
# ==============================
if __name__ == "__main__":
    logger.info("Starting EEG preprocessing for memory condition")
    for sub in SUBJECT_IDS:
        process_into_BIDS(sub, 'mem')

    logger.info("Starting EEG preprocessing for perception condition")
    for sub in SUBJECT_IDS:
        process_into_BIDS(sub, 'perc')

    logger.info("All preprocessing completed.")


2025-11-23 20:39:21,087 - INFO - Starting EEG preprocessing for memory condition
2025-11-23 20:39:21,142 - INFO - Subject ACC014: behavioral data of mem has been loaded and saved as BIDS.
2025-11-23 20:39:21,430 - INFO - Subject ACC014: eeg and time have been loaded.
2025-11-23 20:39:21,701 - INFO - Subject ACC014: eeg and time have been loaded.
2025-11-23 20:39:21,706 - INFO - Get events from subject ACC014 in mem  


Creating RawArray with float64 data, n_channels=30, n_times=217596
    Range : 0 ... 217595 =      0.000 ...   435.190 secs
Ready.


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(
2025-11-23 20:39:22,129 - INFO - Subject ACC015: behavioral data of mem has been loaded and saved as BIDS.
2025-11-23 20:39:22,438 - INFO - Subject ACC015: eeg and time have been loaded.
2025-11-23 20:39:22,694 - INFO - Subject ACC015: eeg and time have been loaded.
2025-11-23 20:39:22,697 - INFO - Get events from subject ACC015 in mem  


Creating RawArray with float64 data, n_channels=30, n_times=214197
    Range : 0 ... 214196 =      0.000 ...   428.392 secs
Ready.


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(
2025-11-23 20:39:23,081 - INFO - Subject ACC016: behavioral data of mem has been loaded and saved as BIDS.
2025-11-23 20:39:23,384 - INFO - Subject ACC016: eeg and time have been loaded.
2025-11-23 20:39:23,666 - INFO - Subject ACC016: eeg and time have been loaded.
2025-11-23 20:39:23,672 - INFO - Get events from subject ACC016 in mem  


...

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(
2025-11-23 20:40:09,359 - INFO - Subject ACC035: behavioral data of perc has been loaded and saved as BIDS.
2025-11-23 20:40:09,676 - INFO - Subject ACC035: eeg and time have been loaded.
2025-11-23 20:40:09,961 - INFO - Subject ACC035: eeg and time have been loaded.
2025-11-23 20:40:09,964 - INFO - Get events from subject ACC035 in perc  


Creating RawArray with float64 data, n_channels=30, n_times=259804
    Range : 0 ... 259803 =      0.000 ...   519.606 secs
Ready.


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(
2025-11-23 20:40:10,399 - INFO - Subject ACC036: behavioral data of perc has been loaded and saved as BIDS.
2025-11-23 20:40:10,762 - INFO - Subject ACC036: eeg and time have been loaded.
2025-11-23 20:40:11,089 - INFO - Subject ACC036: eeg and time have been loaded.
2025-11-23 20:40:11,096 - INFO - Get events from subject ACC036 in perc  


Creating RawArray with float64 data, n_channels=30, n_times=275779
    Range : 0 ... 275778 =      0.000 ...   551.556 secs
Ready.


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_26004/403887070.py:121: RuntimeWarning: Encountered data in "double" format. Converting to float32.
  write_raw_bids(
2025-11-23 20:40:11,550 - INFO - All preprocessing completed.
